1. Предобработка данных: Удалена колонка `Posted On`, которая, вероятно, не влияет на целевую переменную. Пропущенные значения в `Area Locality` заполнены значением 'Unknown'.

2. Разделение на обучающую и тестовую выборки с использованием функции `train_test_split`.

3. Кодирование категориальных признаков и масштабирование числовых с использованием `ColumnTransformer`.

4. Обучение модели с использованием `Pipeline`, включающего предварительную обработку данных и регрессор на основе ближайших соседей с произвольно заданным гиперпараметром K.

5. Оценка качества модели с использованием метрик RMSE и R^2.

6. Подбор гиперпараметра K с использованием `GridSearchCV` и `RandomizedSearchCV`, позволяя проверить, как изменится производительность модели при оптимизации этого параметра.

7. Сравнение исходной модели и модели после оптимизации гиперпараметров.



In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, KFold, StratifiedKFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [18]:
# Загрузка данных
df = pd.read_csv('/content/House_Rent_Dataset.csv')

In [19]:
# Предварительный анализ и предобработка данных
# Например, удалим колонку 'Posted On', так как она, вероятно, не будет полезной для нашей модели
# и заполним пропуски в 'Area Locality' значением 'Unknown'
df.drop(['Posted On'], axis=1, inplace=True)
df['Area Locality'].fillna('Unknown', inplace=True)

In [20]:
# Кодирование категориальных признаков и масштабирование численных
categorical_features = ['Floor', 'Area Type', 'Area Locality', 'City', 'Furnishing Status', 'Tenant Preferred']
numerical_features = ['BHK', 'Size']

In [21]:
# Создадим трансформер колонок
column_transformer = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)])

# Выделяем признаки и целевую переменную
X = df.drop('Rent', axis=1)
y = df['Rent']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [22]:
# Создание и обучение модели KNN
pipeline = Pipeline(steps=[('preprocessor', column_transformer),
                           ('regressor', KNeighborsRegressor(n_neighbors=5))])
pipeline.fit(X_train, y_train)

# Оценка качества модели
y_pred = pipeline.predict(X_test)
print(f"Исходная RMSE: {np.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"Исходный R^2: {r2_score(y_test, y_pred)}")

Исходная RMSE: 52459.005392513376
Исходный R^2: 0.30948960017615346


In [24]:
# Подбор гиперпараметра K с использованием GridSearchCV и RandomizedSearchCV
param_grid = {'regressor__n_neighbors': np.arange(1, 31)}
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
random_search = RandomizedSearchCV(pipeline, param_grid, n_iter=20, cv=5, scoring='neg_mean_squared_error')

grid_search.fit(X_train, y_train)
random_search.fit(X_train, y_train)

# Выводим результаты
print(f"Лучшее количество ближайших соседей (GridSearchCV): {grid_search.best_params_['regressor__n_neighbors']}")
print(f"Лучшее количество ближайших соседей (RandomizedSearchCV): {random_search.best_params_['regressor__n_neighbors']}")

# Оценка качества оптимальной модели
optimal_model = grid_search.best_estimator_
y_pred_optimal = optimal_model.predict(X_test)
print(f"Оптимальная RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_optimal))}")
print(f"Оптимальный R^2: {r2_score(y_test, y_pred_optimal)}")

Лучшее количество ближайших соседей (GridSearchCV): 23
Лучшее количество ближайших соседей (RandomizedSearchCV): 23
Оптимальная RMSE: 41200.02534050018
Оптимальный R^2: 0.5740828912045376
